# Understanding Web Requests 

  * Programming for the web isn't like programming on a single machine. 
    * There are at least two computers involved (many more in advanced setups) 
    * Program flow is based on request/response cycles 
  * In web programming the two parties involved are 
    * *The client*. This is the web browser (e.g. Chrome or Firefox). 
      * The client initiates all requests. 
    * *The server* The server is your program. 
      * The server waits for client requests 
      * The server *validates* and responds to requests 
  * Requests have to be directed somewhere. 
  * The *location* that a request is directed to is a URL 
  * URL is Universal Resource Locator.
  * URLs have four parts

Here's an example URL: 

```
http://www.lifealgorithmic.com/cis-15?arg=value&arg2=value2
```

| URL Part | Meaning | 
| --- ---- | ------- | 
| `http:` | Specifies the *Hypertext Transfer Protocol* should be used to connect to the server. <br> There are many protocols but HTTP is the most common |
| `//www.lifealgorithmic.com` | The *host* or the DNS name of the server to be contacted | 
| `/cis-15` | The *path* of the resource. This is like a file path but the server is free to interpret this any way it wants to. <br> **This part is case sensitive** | 
| `?arg=value&arg2=value2` | Arguments to the path. These are a convention used by GET forms. The server is free to do anything it wants with these | 

  * Web browsers primarily issue two kinds of requests. 
    * `GET` requests load a URL with arguments like the ones in the URL above
      * GET requests reveal their arguments in the browser location bar
      * That makes them a poor choice for forms with sensitive (or a lot) of data 
    * `POST` requests load a URL with embedded arguments. 
      * Embedded arguments are a bit easier to retrieve in a program. 

Here's an example of what the web browser sends the server in a `GET` request: 

```
GET /hello.htm HTTP/1.1
User-Agent: Mozilla/4.0 (compatible; MSIE5.01; Windows NT)
Host: www.tutorialspoint.com
Accept-Language: en-us
Accept-Encoding: gzip, deflate
Connection: Keep-Alive
```

Here's an example of what the web browser sends the server in a `POST` request: 

```
POST /cgi-bin/process.cgi HTTP/1.1
User-Agent: Mozilla/4.0 (compatible; MSIE5.01; Windows NT)
Host: www.tutorialspoint.com
Content-Type: application/x-www-form-urlencoded
Content-Length: length
Accept-Language: en-us
Accept-Encoding: gzip, deflate
Connection: Keep-Alive

licenseID=string&content=string&/paramsXML=string
```

  * For more information about GET and POST requests read this:
    * https://www.w3schools.com/TagS/ref_httpmethods.asp
  * Flask makes it easy to select where a request goes.
  * This selection in web speak is known as //routing// 
    * Which is not the same as the routing you learn about in cis-81

Here's a snippet from this week's sample code:

In [ ]:
@app.route('/result', methods=['POST'])
def result():
    return render_template('result.html')    

@app.route('/words', methods=['POST'])
def do_madlib():
    return render_template('words.html')

@app.route('/')
def start_madlib():
    return render_template('madlib.html')

  * The `@app.route()` *decorator* informs Flask where to send requests. 
  * Here's what they mean:

| Route | Meaning | 
| ----- | ------- | 
| `@app.route('/result', methods=['POST'])` | POST requests that have `/result` in the path <br> GET requests are not allowed |
| `@app.route('/words', methods=['POST'])` | POST requests that have `/words` in the path <br> GET requests are not allowed | 
| `@app.route('/')` | GET requests sent to the root path (`/`). <br> GET is the default, no other request is allowed. | 

  * It's possible to have a route that handles multiple request types. Those appear like this:

In [ ]:
@app.route('/words', methods=['POST', 'GET'])
def do_madlib():
    return render_template('words.html')


# Example Code 

You can find starter code for this week's project here:

In [ ]:
from flask import Flask, request 

app = Flask(__name__) 

start_template = '''
<html>
    <head>
        <title>Enter your Mad Lib</title>
    </head>
    <body>
        <form action="/" method="post">
            
          Enter the madlib with embedded {} marks:<br>
          <input type="text" name="madlib"><br>

          Enter the type of the first word:<br>
          <input type="text" name="word_type_1"><br>

          <input type="submit" value="Submit">
        </form>
    </body>
</html>
'''

words_template = ''' 
<html>
    <head>
        <title>Enter the Words</title>
    </head>
    <body>
        <form action="/result" method="post">
            
          Enter a {word_type_1}:<br>
          <input type="text" name="word_1"><br>

          <input type="hidden" name="madlib" value="{madlib}">
          
          <input type="submit" value="Submit">
        </form>
    </body>
</html>
''' 

result_template = '''
<html>
    <head>
        <title>Enter the Words</title>
    </head>
    <body>
        The completed Mad Lib is:<br>
        <pre>{complete_madlib}</pre>
        <br>
        <a href="/">Start Over</a>
    </body>
</html>
'''

@app.route('/result', methods=['POST'])
def result() :
    madlib = request.form['madlib']
    word1 = request.form['word_1']
    complete = madlib.format(word1)
    return result_template.format(complete_madlib=complete)

@app.route('/', methods=['GET', 'POST'])
def index() :
    if request.method == 'GET' :
        return start_template
    else:
        madlib = request.form['madlib']
        wt1 = request.form['word_type_1']
        return words_template.format(word_type_1 = wt1, madlib = madlib )
        

if __name__ == '__main__' : 
    app.run(host='0.0.0.0', port=8080, debug=True)